In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow #לחילוץ פריים
#cv2.imshow #מי שלא עובד בקולב יכול לעבוד עם הפונקציה הזו מתוך cv2
#ולכתוב:
#cv2_imshow(frame)

from IPython.display import HTML #לצפייה בסרטון
from base64 import b64encode #לצפייה בסרטון

In [ ]:
cap = cv2.VideoCapture("baloon2.mp4") #2 יצירת משתנה סרט

In [ ]:
type(cap)

In [ ]:
#צפייה בסרטון
mp4 = open("baloon2.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
# 3 חילוץ פריים ראשון מהוידאו
ret, frame = cap.read()  # קורא את הפריים הראשון
#cap.release()

In [ ]:
ret, frame =cap.read()

In [ ]:
ret

In [ ]:
# 4 המרת צבעים
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [ ]:
# 5 הצגת התמונה המומרת
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("Original Frame")
plt.grid(True)
plt.xticks(np.arange(0, frame.shape[1], 50), rotation =90)  # כל 50 פיקסלים על ציר X
plt.yticks(np.arange(0, frame.shape[0], 50))  # כל 50 פיקסלים על ציר Y
plt.show()
plt.show()

In [ ]:
# המרה לפי כמה שיטות:
if ret:
    # המרות לצבעים שונים
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)

    # הצגה של 4 גרסאות
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))

    axs[0].imshow(gray, cmap='gray')
    axs[0].set_title("Grayscale")

    axs[1].imshow(hsv)
    axs[1].set_title("HSV")

    axs[2].imshow(lab)
    axs[2].set_title("LAB")

    axs[3].imshow(ycrcb)
    axs[3].set_title("YCrCb")

    for ax in axs:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("somthing went wrong")


In [ ]:
# 6: חיתוך קורדינטות עם התמונה על פי ערכים משלב 5
# הגדרת הקואורדינטות שנבחרו (למשל)
x, y = 275, 525

# הגבלת גבולות החיתוך כדי לא לצאת מהתמונה
h, w = hsv.shape[:2]
crop_size = 10  # חצי מגודל החיתוך (50x50)

x_start = max(0, x - crop_size)
x_end = min(w, x + crop_size)
y_start = max(0, y - crop_size)
y_end = min(h, y + crop_size)

# חיתוך האזור מתוך תמונת HSV (שהומרה קודם)
crop = hsv[y_start:y_end, x_start:x_end]

In [ ]:
# הצגת מה שחתכתי
# המרה מ-HSV ל-RGB לצורך הצגה נכונה
crop_rgb = cv2.cvtColor(crop, cv2.COLOR_HSV2RGB)
# הצגה
plt.imshow(crop_rgb)
plt.axis("off")

In [ ]:
crop.shape

In [ ]:
# נניח ש־crop הוא האזור שחתכת מתוך hsv
pixels = crop.reshape(-1, 3)  # הפיכת המטריצה לרשימת פיקסלים


In [ ]:
pixels = crop.reshape(-1, 3)  # הפיכת המטריצה לרשימת פיקסלים
# חישוב ממוצע, מינימום ומקסימום לכל ערוץ (H, S, V)
mean_hsv = np.mean(pixels, axis=0)
min_hsv = np.min(pixels, axis=0)
max_hsv = np.max(pixels, axis=0)

print("ממוצע HSV:", mean_hsv)
print("מינימום HSV:", min_hsv)
print("מקסימום HSV:", max_hsv)

In [ ]:
# קביעת טווח מעט רחב יותר ממה שהתקבל במינ מקס משיקולי אור\צל וכו'
##צריך להמיר ל int!!!
lower = np.array([
    max(0, int(min_hsv[0]) - 2),     # Hue תחתון - טיפה פחות
    max(0, int(min_hsv[1]) - 20),    # Saturation - גמיש יותר
    max(0, int(min_hsv[2]) - 20)     # Value - גם כאן
])

upper = np.array([
    min(179, int(max_hsv[0]) + 2),   # Hue עליון - טיפה יותר
    min(255, int(max_hsv[1]) + 30),  # Saturation
    min(255, int(max_hsv[2]) + 30)   # Value
])


In [ ]:
upper, lower

In [ ]:
#יצירת מסיכה (mask) שמבליטה רק את הכדור לפי הצבע
# הנחה: hsv_frame הוא הפריים שהומר ל-HSV
mask = cv2.inRange(hsv_frame, lower, upper)

In [ ]:
result = cv2.bitwise_and(frame, frame, mask=mask)

In [ ]:
plt.imshow(mask, cmap="gray")
plt.title("the mask (inRange)")
plt.axis("off")

In [ ]:
#: המרה ל-RGB לצורך הצגה ב-matplotlib
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

plt.imshow(result_rgb)
plt.title("Just the object")
plt.axis("off")

In [ ]:
lower,upper

In [ ]:
# מציאת קונטורים
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# שלב 3: בחר את הקונטור הכי גדול (בהנחה שזה הכדור)
largest_contour = max(contours, key=cv2.contourArea)

# שלב 4: צייר את הקונטור על עותק של התמונה המקורית
frame_with_contour = frame.copy()
cv2.drawContours(frame_with_contour, [largest_contour], -1, (0, 255, 0), 2)  # ירוק

# המרה ל-RGB לצורך הצגה עם matplotlib
frame_rgb = cv2.cvtColor(frame_with_contour, cv2.COLOR_BGR2RGB)
# שלב 5: הצגה
plt.imshow(frame_rgb)
plt.axis("off")
plt.show()


In [ ]:
#מציאת מרכז הקונטור
M = cv2.moments(largest_contour)

In [ ]:
#חישוב המרכז
center_x = int(M["m10"] / M["m00"])
center_y = int(M["m01"] / M["m00"])
center = (center_x, center_y)


# ציור עיגול במרכז
frame_with_center = frame.copy()
cv2.circle(frame_with_center, center, 5, (0, 0, 255), -1)  # עיגול אדום

# ציור גם של הקונטור מסביב לכדור (כמו קודם)
cv2.drawContours(frame_with_center, [largest_contour], -1, (0, 255, 0), 2)  # ירוק

# הצגה
plt.imshow(cv2.cvtColor(frame_with_center, cv2.COLOR_BGR2RGB))
plt.title("Center(Red) and contour (Green)")
plt.show()

In [ ]:
# יצירת Kalman Filter בגודל 4x2:
# מצב = [x, y, dx, dy] ← מיקום ומהירות
# מדידה = [x, y]
kalman = cv2.KalmanFilter(4, 2)

# מטריצת המעבר (transition) ← איך המצב מתפתח עם הזמן
kalman.transitionMatrix = np.array([
    [1, 0, 1, 0],  # x ← x + dx
    [0, 1, 0, 1],  # y ← y + dy
    [0, 0, 1, 0],  # dx ← dx
    [0, 0, 0, 1]   # dy ← dy
], dtype=np.float32)

# מטריצת מדידה ← איך המצב משפיע על המדידה בפועל
kalman.measurementMatrix = np.array([
    [1, 0, 0, 0],  # מודדים רק x
    [0, 1, 0, 0]   # מודדים רק y
], dtype=np.float32)

# רעש מדידה ורעש תנועה
#קלמן סומך פחות על התחזית ככל שהערך גבוה. מטריצה  3 על 3 עם 1 באלכסון רעש בתנועה
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
# ככל שהערך גבוה קלמן סמך פחות על הזיהוי רעש במדידה
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1

# נגדיר את המצב ההתחלתי כמרכז הקונטור שמצאנו (משלב 13)
kalman.statePre = np.array([[np.float32(center[0])],
                            [np.float32(center[1])],
                            [0], [0]])


In [ ]:
initialized = False
frames = []

cap.release()
cap = cv2.VideoCapture("baloon2.mp4")

# נקרא 30 פריימים ראשונים לדוגמה
for _ in range(30):
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = np.array([[np.float32(cx)], [np.float32(cy)]])

            if not initialized:
                kalman.statePre = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
                initialized = True

            prediction = kalman.predict()
            pred_x = int(prediction[0][0])
            pred_y = int(prediction[1][0])

            kalman.correct(center)

            # ציור
            cv2.drawContours(frame, [largest], -1, (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)  # אדום – מיקום אמיתי
            cv2.circle(frame, (pred_x, pred_y), 10, (255, 0, 0), -1)  # כחול – תחזית

    # שמירה להצגה
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

cap.release()

# הצגה של 6 פריימים לדוגמה
plt.figure(figsize=(15, 10))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(frames[i])
    plt.title(f"Frame {i + 1}")
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
cap.release()
cap = cv2.VideoCapture("baloon2.mp4")

initialized = False
frame_index = 0
display_index = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = np.array([[np.float32(cx)], [np.float32(cy)]])

            if not initialized:
                kalman.statePre = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
                initialized = True

            prediction = kalman.predict()
            kalman.correct(center)

            # ציור תמיד – גם אם לא נשמר
            cv2.drawContours(frame, [largest], -1, (0, 255, 0), 2)  # ירוק
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)         # אדום
            cv2.circle(frame, (int(prediction[0][0]), int(prediction[1][0])), 10, (255, 0, 0), 2)  # כחול

    # רק כל 50 פריימים – נשמור ונציג
    if frame_index % 25 == 0:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6, 4))
        plt.imshow(rgb)
        plt.title(f"Frame {frame_index}")
        plt.axis("off")
        plt.show()

    frame_index += 1

cap.release()


In [ ]:
#LinearRegression First Frames
from sklearn.linear_model import LinearRegression

# פתיחת הווידאו
cap = cv2.VideoCapture("baloon2.mp4")

# טווח HSV לכתום
lower = np.array([6, 100, 160])
upper = np.array([17, 230, 255])

# איסוף דאטה
X_frames = []
y_positions = []

frame_index = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            X_frames.append([frame_index])
            y_positions.append([cx, cy])

    frame_index += 1

cap.release()

# המרה ל־NumPy
X = np.array(X_frames)
y = np.array(y_positions)

# אימון רגרסיה רק אם יש דאטה
if len(y) > 0:
    model_x = LinearRegression().fit(X, y[:, 0])
    model_y = LinearRegression().fit(X, y[:, 1])

    # חיזוי
    y_pred_x = model_x.predict(X)
    y_pred_y = model_y.predict(X)

    # פתיחה מחדש של הסרטון
    cap = cv2.VideoCapture("baloon2.mp4")
    frame_index = 0
    displayed = 0

    plt.figure(figsize=(15, 10))
    while displayed < 6:
        ret, frame = cap.read()
        if not ret or frame_index >= len(y_pred_x):
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                pred_x = int(y_pred_x[frame_index])
                pred_y = int(y_pred_y[frame_index])

                cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)  # אדום
                cv2.circle(frame, (pred_x, pred_y), 10, (255, 0, 0), 2)  # כחול

                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                plt.subplot(2, 3, displayed + 1)
                plt.imshow(rgb)
                plt.title(f"Frame {frame_index}")
                plt.axis("off")
                displayed += 1

        frame_index += 1

    cap.release()
    plt.tight_layout()
    plt.show()
else:
    print("something went wrong")

In [ ]:
#LinearRegression every 25 frames
# פתיחה מחדש של הסרטון
cap = cv2.VideoCapture("baloon2.mp4")
frame_index = 0

while True:
    ret, frame = cap.read()
    if not ret or frame_index >= len(y_pred_x):
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            pred_x = int(y_pred_x[frame_index])
            pred_y = int(y_pred_y[frame_index])

            # ציור
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            cv2.circle(frame, (pred_x, pred_y), 10, (255, 0, 0), 2)

            # רק כל 25 פריימים
            if frame_index % 25 == 0:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(6, 4))
                plt.imshow(rgb)
                plt.title(f"Frame {frame_index}")
                plt.axis("off")
                plt.show()

    frame_index += 1

cap.release()
